<a href="https://colab.research.google.com/github/stepkurniawan/stock-probabilistic-model/blob/main/2022_06_22_MCMC_simu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  2 12:56:02 2022

@author: steph

Our simulation consists of 2 stocks 
we simulate their drift, volatility, and transition matrix
and plug them in max likelihood
before testing using MCMCs
"""

import numpy as np
import pandas as pd

##############################################################################
### VARIABLES ###
# drift = normal distribution(mean, sigma)
# dim(drift) = [stock] [state] 
# list of mean of stocks : [stock1, stock2]
mean_stocks = [0,0]
# list of sigma of stocks : [stock1, stock2]
sigma_stocks = []
sigma_states_elements = [0.5, 0.2, 0.5]

# randomly generated 2 stocks
num_stocks = 2

# we have 3 states: 
# state 0: crisis, state 1: stable, state 2: bubble
num_state = 3

# how many time (index of time)
N = 50

#fix the random seed
# np.random.seed(0)
#############################################################################
# b : drift for each stocks, for each state
# ex: b[0,1] is drift for stock 0 and state 1
b = np.random.randn(num_stocks, num_state)

# b0 < b1 < b2
for j in range(num_stocks):
    for i in range(num_state):
        if i == 0:
            b[j,i] = np.random.randn(1)
        else: 
            # make sure b of the next state is larger than the one before
            # b0 (crisis period) < b1 (stable) < b2 (bubble periode)
            b[j,i] = np.random.normal(b[j,i-1] +1, 0.5,1) 


print("drift matrix",b)

drift matrix [[ 0.49574953  2.25368764  3.86062002]
 [-1.08263512 -0.47500955 -0.08019348]]


In [ ]:
# volatility
sigma_stocks = [
    np.full((num_stocks, num_stocks), sigma_states_elements[x], dtype=np.float32)
    for x in range(num_state)
]
for x in sigma_stocks:
    np.fill_diagonal(x, 1)

sigma_stocks

[array([[1. , 0.5],
        [0.5, 1. ]], dtype=float32), array([[1. , 0.2],
        [0.2, 1. ]], dtype=float32), array([[1. , 0.5],
        [0.5, 1. ]], dtype=float32)]

In [ ]:
# create 3 matrix of dxd matrix
# for state 1 & 3 -> similar covariance matrix

# transition matrixs 
# P [stock][from state][to state]
P = [[0.3, 0.4, 0.3], [0.1, 0.9, 0.0], [0.4, 0.3, 0.3]], \
    [[0.2, 0.6, 0.2], [0.01, 0.9, 0.09], [0.4, 0.3, 0.3]]

P

([[0.3, 0.4, 0.3], [0.1, 0.9, 0.0], [0.4, 0.3, 0.3]],
 [[0.2, 0.6, 0.2], [0.01, 0.9, 0.09], [0.4, 0.3, 0.3]])

In [ ]:
# simulate y : state matrix
# t = 0 : first state : random between state 1,2,3
# y[i,t] : stock i, and t time
# ex: y[0,1] : state at stock 0 when time is in index 1.  

y = np.zeros((num_stocks, N))

for i in range(num_stocks):
    for t in range(N):
        if t == 0:
            y[i,t] = np.random.randint(0,3)
        else: 
            prev_state = int(y[i, t-1])
            y[i,t] = np.random.choice( np.arange(0,3), p=[P[i][prev_state][0], \
                                                          P[i][prev_state][1], \
                                                          P[i][prev_state][2]] )
y = y.astype('int') # state is always integer
print("state matrix", y)

state matrix [[2 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 0
  0 0 0 1 1 1 1 1 1 0 1 1 1 1]
 [2 0 1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 2 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 2 1 1 1 1 1]]


In [ ]:
# DELTA of BROWNIAN MOTION is a normal distribution of mean 0 and std 1
# 1 stock have 1 brownian motion
# the length of 1 brownian motion is N
# brownian motion have dim(num_stock, N)

# brownian_motion_delta = np.zeros((num_stocks, N))

# for j in range(num_stocks): 
#     brownian_motion_delta[j] = np.random.standard_normal(N)

brownian_motion_delta = np.array([np.random.standard_normal(N) for _ in range(num_stocks)])
brownian_motion_delta

array([[-0.40213111, -0.10732894, -0.08980222,  0.74169346,  1.19198356,
         0.89402974, -0.75396802, -0.26404059, -1.42219957,  0.13038717,
        -0.85128952, -1.00970655, -0.12759106, -1.25791062,  0.15212076,
        -1.18325653,  1.32573547, -1.71982518,  0.73218598, -1.5483139 ,
         1.01372726, -0.21899541, -1.04881343,  0.72970982,  0.17840918,
         0.28760359, -0.16039726, -1.15660443,  1.44733044,  0.82698816,
        -0.62206343,  0.58578879, -0.04539203,  2.11228508, -0.9094792 ,
         0.77113651, -1.18304834,  0.11673322, -0.03302903, -0.40931618,
        -1.47528989,  0.36047537, -0.08795342, -1.66552235, -1.60555524,
        -0.73275546,  0.36592997,  0.0353362 , -0.11507964,  0.61741499],
       [ 0.65378073,  0.98004766, -2.36768845, -0.40866582,  0.51378666,
        -0.49132414,  2.14273857,  0.20007113,  1.03569708, -2.15296979,
         1.34979504, -0.2755222 , -2.94427995,  0.86420184, -0.43579463,
         1.21767785,  0.44529785,  0.91467253, -0.

In [ ]:
# r : rate of Return of stock price from timme 0 to time t
r = np.zeros((num_stocks, N))

for i in range(num_stocks):
    for n in range(1,N+1):
        sum_b = 0
        sum_sigma = 0
        for t in range(n):
            state_now = y[i,t]
            sum_b = sum_b + b[i , state_now]
            # Toby: what's the difference of d and i?
            for d in range(num_stocks):
                sum_sigma = sum_sigma + (sigma_stocks[state_now][i][d] * brownian_motion_delta[i, t])
            r[i,t] = sum_b + sum_sigma

print("return matrix", r)


return matrix [[  3.25742335   3.59217947   5.73810444   8.88182423  11.16554911
   14.49207244  15.84099844  17.77783737  18.32488552  20.73503776
   21.96717796  23.00921774  25.10979611  25.853991    28.29022354
   29.12400334  32.96857355  33.15847097  36.29078179  36.68649274
   40.1566531   42.14754624  43.14265776  44.73297202  45.49633533
   48.09514727  50.15635819  51.02212051  55.01260467  58.2586781
   59.76588961  61.14032233  63.33953953  68.12796927  67.25950001
   68.9119543   67.63313132  68.30398069  68.75018668  70.5126949
   70.99603466  73.68229274  75.83043627  76.08549708  76.41251841
   75.80913475  78.50193836  80.79802943  82.9136215   85.90820713]
 [  0.90047762   1.28791399  -2.02832171  -2.99373024  -2.8521958
   -3.91679432  -0.78287993  -0.56296671   0.91038544  -2.14818787
   -1.00344338  -1.80907957  -5.81722506  -5.25519241  -6.25315552
   -5.26695164  -5.20760377  -3.91578846  -4.13621372  -7.74229413
   -6.93524248  -7.93156362 -10.09682947  -7.92973

In [ ]:
# Price matrix S
# dimension: number_of_stock x N times
# ex: S[0,1] : stock 0 , time 1
S = np.zeros((num_stocks, N))
for i in range(num_stocks):
    for n in range(N):
        if n == 0:
            S[i,n] = np.random.randint(50,100)
        else:
            S[i,n] = S[i, n-1]*r[i,n]
print("price matrix", S)

price matrix [[ 8.40000000e+01  3.01743076e+02  1.73143328e+03  1.53782861e+04
   1.71707009e+05  2.48839041e+06  3.94185885e+07  7.00777256e+08
   1.28416630e+10  2.66272367e+11  5.84925248e+12  1.34586724e+14
   3.37944520e+15  8.73721457e+16  2.47177753e+18  7.19880571e+19
   2.37334356e+21  7.86964434e+22  2.85595545e+24  1.04774989e+26
   4.20741289e+27  1.77332129e+29  7.65057936e+30  3.42233153e+32
   1.55703543e+34  7.48858482e+35  3.75600142e+37  1.91639157e+39
   1.05425692e+41  6.14196145e+42  3.67079790e+44  2.24433767e+46
   1.42155315e+48  9.68475290e+49  6.51391638e+51  4.48886708e+53
   3.03596137e+55  2.07368246e+57  1.42566057e+59  1.00527168e+61
   7.13703034e+62  5.25872759e+64  3.98771607e+66  3.03407360e+68
   2.31841204e+70  1.75756811e+72  1.37972503e+74  1.11479064e+76
   9.24313292e+77  7.94060977e+79]
 [ 5.30000000e+01  6.82594416e+01 -1.38452107e+02  4.14488261e+02
  -1.18220168e+03  4.63044080e+03 -3.62507919e+03  2.04079891e+03
   1.85791361e+03 -3.9911474

In [ ]:
# MCMC PyMC

from pymc3 import Model, Normal, Uniform, Exponential, LKJCholeskyCov, Dirichlet, DirichletMultinomial
from pymc3 import sample


with Model() as test_model:
    
    μ = Normal('μ', mu=0, sd=10)
    σ = Uniform('σ', 0, 10)
    b1 = Normal('b1', mu = 0, sd=10)
    b2 = Normal('b2', mu = 0, sd=10)
    b3 = Normal('b3', mu = 0, sd=10)
    sigma1 = Normal('sigma1', mu = 0, sd=10)
    sigma2 = Normal('sigma2', mu = 0, sd=10)
    sigma3 = Normal('sigma3', mu = 0, sd=10)
    sd_dist = Exponential.dist(1.0)
    # chol, corr, sigmas = LKJCholeskyCov(
    #     'covariance_mat', eta=4, n=10, sd_dist=sd_dist, compute_corr=False
    # )
    P = DirichletMultinomial('P', n=0.33, a=2, shape=(3,3) )
    y0 = Uniform('y0', 1,3)
    y = Uniform('y', 1,3)
    
with test_model:
    
    dist = Normal('dist', mu=μ, sd=σ, observed=S)
    samples = sample(1000, tune=1000, cores=4, random_seed=42)
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [y, y0, sigma3, sigma2, sigma1, b3, b2, b1, σ, μ]
>Metropolis: [P]


/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/hmc/quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/hmc/quadpotential.py:203: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/hmc/quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/hmc/quadpotential.py:203: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/hmc/quadpotential.py:224: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/usr/local/lib/python3.7/dist-packages/pymc3/step_methods/hmc/quadpot

RuntimeError: ignored